In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import os

/Users/sarthak/Documents/Playground/RecipeApp/RecipeAppVenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_and_sample_dataset(input_csv, sample_ratio=0.01):
    # Load dataset
    df = pd.read_csv(input_csv)
    # Sample 1% of the dataset randomly
    sample_df = df.sample(frac=sample_ratio)
    # Save the sampled dataset to a new CSV
    sample_csv = "sample_set.csv"
    sample_df.to_csv(sample_csv, index=False)
    return sample_csv

In [5]:
input_csv = "RecipeNLG_dataset.csv"  # Change this to your dataset's filename
sample_ratio = 0.01  # 1% of the data
sample_csv = load_and_sample_dataset(input_csv, sample_ratio)

In [13]:
def embed_data_with_sentence_transformers(csv_file, model_name, file_column):
    # Load the sample dataset
    df = pd.read_csv(csv_file)
    # Assuming the text to be embedded is in the 'text' column
    texts = df[file_column].tolist()
    # Load the model
    model = SentenceTransformer(model_name)
    #Put the model on MPS
    model = model.to(torch.device("mps"))
    # Generate embeddings
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings


def save_embeddings_to_csv(embeddings, output_file):
    # Convert the embeddings into a DataFrame
    embeddings_df = pd.DataFrame(embeddings)
    # Save the DataFrame to a CSV file
    embeddings_df.to_csv(output_file, index=False)
    print(f"Embeddings saved to {output_file}")


In [14]:
ingredients_embedings = embed_data_with_sentence_transformers(csv_file = "sample_set.csv", model_name='all-MiniLM-L6-v2', file_column ='ingredients')
save_embeddings_to_csv(embeddings = ingredients_embedings, output_file = "IngredientsEmbeddings.csv")

KeyError: 'text'

In [1]:

def load_and_sample_dataset(input_csv, sample_ratio=0.01):
    # Load dataset
    df = pd.read_csv(input_csv)
    # Sample 1% of the dataset randomly
    sample_df = df.sample(frac=sample_ratio)
    # Save the sampled dataset to a new CSV
    sample_csv = "sample_set.csv"
    sample_df.to_csv(sample_csv, index=False)
    return sample_csv

def embed_data_with_sentence_transformers(csv_file, model_name='all-MiniLM-L6-v2'):
    # Load the sample dataset
    df = pd.read_csv(csv_file)
    # Assuming the text to be embedded is in the 'text' column
    texts = df['text'].tolist()
    # Load the model
    model = SentenceTransformer(model_name)
    # Generate embeddings
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings

def create_faiss_index(embeddings):
    # Dimension of the vectors
    d = embeddings.shape[1]
    # Creating a FAISS index
    # Here we use IndexFlatL2 which is good for general purposes. 
    # You might want to explore other indices depending on your specific needs.
    index = faiss.IndexFlatL2(d)
    # FAISS expects vectors in float32
    embeddings = embeddings.astype('float32')
    # Adding vectors to the index
    index.add(embeddings)
    # Save the index
    faiss.write_index(index, "faiss_index.idx")


RecipeApp/     RecipeAppVenv/


In [ ]:
def main():
    input_csv = "your_dataset.csv"  # Change this to your dataset's filename
    sample_ratio = 0.01  # 1% of the data
    sample_csv = load_and_sample_dataset(input_csv, sample_ratio)
    embeddings = embed_data_with_sentence_transformers(sample_csv)
    create_faiss_index(embeddings)
    print("Sample set created, embeddings generated, and FAISS index created.")

if __name__ == "__main__":
    main()
